In [1]:
import csv

csv_file = open('Cards.csv', 'r') 
reader = csv.reader(csv_file, delimiter=';')
devices = {}
next(reader)
total_cards = 0
max_card_temp = 0
hottest_device = ''

for device, card, temp in reader:
    temp = int(temp)
    if device not in devices:
        devices[device] = {'total_of_cards': 1, 'high_temp_cards': 0, 'max_temp': temp, 'total_temp': [temp], 'hottest_card_name': card}
    else:
        devices[device]['total_of_cards'] += 1
        if temp > 70:
            devices[device]['high_temp_cards'] += 1
            devices[device]['hottest_card_name'] = card  
        if temp > devices[device]['max_temp']:
            devices[device]['max_temp'] = temp           
        devices[device]['total_temp'].append(temp)

for device, dt in devices.items():
    total_cards += dt['total_of_cards']
    avg_temp = sum(dt['total_temp'])/len(dt['total_temp'])
    devices[device]['avg_temp'] = avg_temp
    if dt['max_temp'] > max_card_temp:
        max_card_temp = dt['max_temp']
        hottest_device = device

total_devices = len(devices)
hottest_card = devices[hottest_device]['hottest_card_name']  

html_file = open('report.html', 'w')
html_file.write('<html>\n')
html_file.write('<head>\n')
html_file.write('<style>\n')
html_file.write('table, th, td {\n')
html_file.write('  border: 2px solid black;\n')
html_file.write('  border-collapse: collapse;\n')
html_file.write('}\n')
html_file.write('</style>\n')
html_file.write('</head>\n')
html_file.write('<body>\n')
html_file.write('<h1>Summary</h1>\n')
html_file.write('<table>\n')
html_file.write('<tr><th>Total Devices</th><th>Total Cards</th><th>Max Card Temperature</th><th>Hottest Card / Device</th></tr>\n')
html_file.write(f'<tr><td>{total_devices}</td><td>{total_cards}</td><td>{max_card_temp}</td><td>{hottest_card} {hottest_device}</td></tr>\n')
html_file.write('</table>\n')
html_file.write('<h1>Devices</h1>\n')
html_file.write('<table>\n')
html_file.write('<tr><th>Device</th><th>Total # of Cards</th><th>High Temp. Cards #</th><th>Max. Temperature</th><th>Avg. Temperature</th></tr>\n')
for device, dt in devices.items():
    a = dt['total_of_cards']
    b = dt['high_temp_cards']
    c = dt['max_temp']
    d = dt['avg_temp']
    html_file.write(f'<tr><td>{device}</td><td>{a}</td><td>{b}</td><td>{c}</td><td>{d}</td></tr>\n')
html_file.write('</table>\n')
html_file.write('</body>\n')
html_file.write('</html>\n')

csv_file.close()
html_file.close()


In [1]:
# Read server type file
with open('devices.csv', 'r') as f:
    server_type_data = [line.strip().split(',') for line in f.readlines()]

# Read object ID file
with open('pon_ports.csv', 'r') as f:
    object_id_data = [line.strip().split(':') + line.strip().split(',') for line in f.readlines()]

# Create server type dictionary
server_type_dict = {}
for line in server_type_data:
    server_type_dict[line[0]] = line[1]

# Create object ID dictionary
object_id_dict = {}
for line in object_id_data:
    server_name = line[0]
    board_name = line[1]
    pon_type = line[2]
    active_count = int(line[3])
    max_capacity = int(line[4])
    if server_name not in object_id_dict:
        object_id_dict[server_name] = {}
    if board_name not in object_id_dict[server_name]:
        object_id_dict[server_name][board_name] = {}
    if pon_type not in object_id_dict[server_name][board_name]:
        object_id_dict[server_name][board_name][pon_type] = {'active_count': 0, 'max_capacity': 0}
    object_id_dict[server_name][board_name][pon_type]['active_count'] += active_count
    object_id_dict[server_name][board_name][pon_type]['max_capacity'] += max_capacity

print(object_id_dict)

# Calculate PON Utilization
pon_utilization = {}
for server_name in object_id_dict:
    for board_name in object_id_dict[server_name]:
        max_capacity = sum([object_id_dict[server_name][board_name][pon_type]['max_capacity'] for pon_type in object_id_dict[server_name][board_name]])
        active_count = sum([object_id_dict[server_name][board_name][pon_type]['active_count'] for pon_type in object_id_dict[server_name][board_name]])
        pon_utilization[(server_name, board_name)] = {'PON Utilization %': round(active_count/max_capacity*100, 2)}

# Sort and print PON Utilization
sorted_pon_utilization = sorted(pon_utilization.items(), key=lambda x: x[1]['PON Utilization %'], reverse=True)
print('Server Name\tBoard Name\tPON Utilization %')
for key, value in sorted_pon_utilization:
    print(key[0] + '\t' + key[1] + '\t' + str(value['PON Utilization %']) + ' %')

# Calculate PON Count
pon_count = {}
for server_name in object_id_dict:
    for pon_type in server_type_dict:
        count = sum([object_id_dict[server_name][board_name][pon_type]['active_count'] for board_name in object_id_dict[server_name] if pon_type in object_id_dict[server_name][board_name]])
        if server_type_dict[server_name] not in pon_count:
            pon_count[server_type_dict[server_name]] = {}
        if server_name not in pon_count[server_type_dict[server_name]]:
            pon_count[server_type_dict[server_name]][server_name] = {}
        pon_count[server_type_dict[server_name]][server_name][pon_type] = count

# Print PON Count
print('\nServer Type\tServer Name\tPON Type\tPon Count')
for server_type in pon_count:
    for server_name in pon_count[server_type]:
        for pon_type


# Create a dictionary to store PON counts
pon_count = {}

# Loop through the PON data
for line in pon_data:
    server_board_pon, pon_info = line.split(':')
    server, board, pon = server_board_pon.split(',')
    pon_type, active_count, max_capacity = pon_info.split(',')
    active_count = int(active_count)
    max_capacity = int(max_capacity)
    
    # Calculate PON utilization
    pon_utilization = active_count / max_capacity
    
    # Add to PON utilization table
    if server not in pon_utilization_table:
        pon_utilization_table[server] = {}
    if board not in pon_utilization_table[server]:
        pon_utilization_table[server][board] = []
    pon_utilization_table[server][board].append(pon_utilization)
    
    # Add to PON count table
    server_type = server_type_dict[server]
    if server_type not in pon_count:
        pon_count[server_type] = {}
    if server not in pon_count[server_type]:
        pon_count[server_type][server] = {}
    if pon_type not in pon_count[server_type][server]:
        pon_count[server_type][server][pon_type] = 0
    pon_count[server_type][server][pon_type] += 1

# Print PON Count
print('\nServer Type\tServer Name\tPON Type\tPon Count')
for server_type in pon_count:
    for server_name in pon_count[server_type]:
        for pon_type in pon_count[server_type][server_name]:
            pon_count_str = str(pon_count[server_type][server_name][pon_type])
            print(server_type + '\t' + server_name + '\t' + pon_type + '\t' + pon_count_str)

FileNotFoundError: [Errno 2] No such file or directory: 'server_type_file.csv'